In [ ]:
import juno.junodb as junodb
import juno.junoutils as junoutils
import juno.junoml as junoml
import json
import datetime
import pandas as pd
import numpy as np

In [ ]:
with open('../python/pipeline/creds.json') as json_data:
    creds = json.load(json_data)

db = junodb.Database(creds)

In [ ]:
datetime.timedelta(seconds=60*10)

In [ ]:
events = db.getUserEvents(email='royer.8383@gmail.com')

In [ ]:
events = db.getUserEvents(email='royer.8383@gmail.com')
e = events.sort_values(by=['created'])
e = events[['created','eventLabel','eventAction','eventCategory']]
e['ca'] = e.eventCategory + "_" + e.eventAction
e['cal'] = e.eventCategory + "_" + e.eventAction + "_" + e.eventLabel
e = e.sort_values(by=['created'])
e['created'] = pd.to_datetime(e['created'])
e['last_event_time'] = e.created.shift(1)
e = e[e.last_event_time.isnull() == False]
e['since_last_event'] = e['created']-e['last_event_time']
e['since_last_event'] = e['since_last_event'].astype(int)
e['since_last_event'] = e['since_last_event']*1e-9

interac_requests = e[(e['eventCategory'] == 'interac') & (e['eventAction'] == 'request')]
interac_requests

examples = []

for time in interac_requests.created.values:
    
    now = time

    ten_mins_ago = now - np.timedelta64(10*60,'s')
    
    x = e[(e.created >= ten_mins_ago) & (e.created <= now)]
    
    examples.append(x)
    
frauds = []
    
for ex in examples:

    single = ex
    categorial = ['eventLabel', 'eventAction', 'eventCategory', 'ca','cal']
    numerical = ['since_last_event']
    expanded = pd.get_dummies(columns=categorial, data=single, drop_first=False)
    expanded.drop(['created','last_event_time'], axis=1, inplace=True)

    time_since = expanded.since_last_event.describe()
    time_since_lables = 'since_last_'+time_since.index
    time_since_values = time_since.values

    categorical = expanded.drop('since_last_event', axis=1)
    categorical = categorical.sum()
    categorical_labels = categorical.index
    categorical_values = categorical.values

    labels = list(time_since_lables)+list(categorical_labels)
    values = list(time_since_values)+list(categorical_values)

    record = dict(zip(labels, values))

    frauds.append(record)
    
frauds = pd.DataFrame(frauds)

frauds = frauds.fillna(0)

frauds

In [ ]:
single = examples[0]

In [ ]:
all_events = pd.read_csv('./pipeline/data/events_raw.csv')

In [ ]:
a = all_events[(all_events['eventCategory'] == 'interac') & (all_events['eventAction'] == 'request')]
a = a[['metadata.email','created','eventLabel','eventAction','eventCategory']]
a['email'] = a['metadata.email']
a

In [ ]:
def summarizeInteracRequests(email, events):
    
    events['email'] = junoutils.combineColumnsByPrefix(df=events, prefixlist=['email'])
    e = events[events.email == email]
    e = events.sort_values(by=['created'])
    e = events[['created','eventLabel','eventAction','eventCategory']]
    e['ca'] = e.eventCategory + "_" + e.eventAction
    e['cal'] = e.eventCategory + "_" + e.eventAction + "_" + e.eventLabel
    e = e.sort_values(by=['created'])
    e['created'] = pd.to_datetime(e['created'])
    e['last_event_time'] = e.created.shift(1)
    e = e[e.last_event_time.isnull() == False]
    e['since_last_event'] = e['created']-e['last_event_time']
    e['since_last_event'] = e['since_last_event'].astype(int)
    e['since_last_event'] = e['since_last_event']*1e-9

    interac_requests = e[(e['eventCategory'] == 'interac') & (e['eventAction'] == 'request')]
    interac_requests

    examples = []

    for time in interac_requests.created.values:

        now = time

        ten_mins_ago = now - np.timedelta64(10*60,'s')

        x = e[(e.created >= ten_mins_ago) & (e.created <= now)]

        examples.append(x)

    requests = []

    for ex in examples:

        single = ex
        categorial = ['eventLabel', 'eventAction', 'eventCategory', 'ca','cal']
        numerical = ['since_last_event']
        expanded = pd.get_dummies(columns=categorial, data=single, drop_first=False)
        expanded.drop(['created','last_event_time'], axis=1, inplace=True)

        time_since = expanded.since_last_event.describe()
        time_since_lables = 'since_last_'+time_since.index
        time_since_values = time_since.values

        categorical = expanded.drop('since_last_event', axis=1)
        categorical = categorical.sum()
        categorical_labels = categorical.index
        categorical_values = categorical.values

        labels = list(time_since_lables)+list(categorical_labels)
        values = list(time_since_values)+list(categorical_values)

        record = dict(zip(labels, values))
        record['email'] = email

        requests.append(record)

    return requests

In [ ]:
a = all_events[(all_events['eventCategory'] == 'interac') & (all_events['eventAction'] == 'request')]
a = a[['metadata.email','created','eventLabel','eventAction','eventCategory']]
a['email'] = a['metadata.email']
a = a[a['email'].isnull() == False]

In [ ]:
len(a.email.unique())

In [ ]:
all_requests = []

for email in a.email.unique():
    all_requests = all_requests + summarizeInteracRequests(email, a)
    
results = pd.DataFrame(all_requests)
results

In [ ]:
for col in sorted(results.columns):
    print(col)

In [ ]:
sub = results[['email','since_last_mean','ca_interac_request']].sort_values(by='ca_interac_request', ascending=False)
sub[sub.email.isin(fraudsters)]

In [ ]:
results = results.fillna(0)
results = results[['email'] + list(set(results.columns)-set(['email']))]
results

In [ ]:
fraudsters = ['gaelkevin@hotmail.com','royer.8383@gmail.com','adventurous7381@gmail.com']

In [ ]:
results['fraud'] = results.email.isin(fraudsters)

In [ ]:
results[results.fraud == True]

In [ ]:
num = results.drop(['email','fraud'], axis=1)

num = junoutils.scaleDf(num)

In [ ]:
auto = junoml.train_autoencoder(df=num, fraud_column=results.fraud)

autoencoder_predictions = junoml.autoencoder_prediction(autoencoder=auto, scaled_data=num)

In [ ]:
autoencoder_predictions['email'] = results.email

In [ ]:
x = autoencoder_predictions.sort_values(by='biggest_anomaly_score_autoencoder', ascending=False)

In [ ]:
x

In [ ]:
x[x.email.isin(fraudsters)]

In [ ]:
ifst = junoml.train_isolation_forest(data=num, contamination=0.01)

In [ ]:
ifst = junoml.isolation_forest_predictions(data=num, isolation_forest=ifst)

In [ ]:
results.shape

In [ ]:
results['isolation_forest_anomaly'] = ifst

In [ ]:
pd.DataFrame(results.groupby('email')['isolation_forest_anomaly'].sum()).reset_index().sort_values(by='isolation_forest_anomaly', ascending=False)